In [49]:
import pickle
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from inference_utils import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

In [3]:
with open('data/general/estonian-stopwords.txt') as file:
    lines = [line.rstrip() for line in file]
    stopwords_est = lines

In [5]:
stopwords_est[stopwords_est.conta]

'teistsugusega'

In [13]:
models = {}

dir = 'models/column/'
models['column'] = pickle.load(file = open(dir + 'best_model_rfc_column_2023-03-24_16-04-33_0.6658043781807724.pkl', 'rb'))
dir = 'models/text/'
models['text'] = pickle.load(file = open(dir + 'best_model_rfc_text_2023-03-24_17-12-20_0.5555078639169283.pkl', 'rb'))

data = {}
data['column'] = pd.read_csv('data/inference/AM_ETMM_column.csv', index_col='id')
data['text'] = text_to_bow(path='data/inference/AM_ETMM_text.csv')

/home/till/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.3 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/till/.local/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.1.3 when using version 1.1.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/till/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['aidu', '

In [14]:
results = pd.DataFrame()
results['id'] = data['column'].index
results.set_index('id', inplace=True)

for key, model in models.items():
    results[key] = [[-1]] * len(results)

results['safe'] = [-1] * len(results)

In [15]:
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

for key, model in models.items():
    current_data = data[key]
    model_features = model.feature_names_in_
    data_features = current_data.columns

    # align features
    model_not_data = [feature for feature in model_features if feature not in data_features]
    current_data[model_not_data] = 0
    intersection = [feature for feature in current_data.columns if feature in model_features]
    current_data = current_data[intersection]
    current_data = current_data[model_features]
    data[key] = current_data   


852
2000


In [16]:
for key in models.keys():
    for i,item in enumerate(models[key].predict_proba(data[key])):
        id = data[key].iloc[i].name
        results[key].loc[id] = np.array(item)

/tmp/ipykernel_47891/2038371742.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results[key].loc[id] = np.array(item)
/tmp/ipykernel_47891/2038371742.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results[key].loc[id] = np.array(item)


In [52]:
# !!! make sure models have seen same labels
results['prediction'], results['certainty'] = zip(*results.apply(lambda row: vote([row.column, row.text]), axis=1))

In [53]:
type_lookup = pd.read_csv('data/inference/type_lookup.csv', index_col=0)
results.prediction = results.prediction.replace(type_lookup.id_est.to_list(), type_lookup.estonian.to_list())

# replace if label already in text
a = results.prediction.copy().tolist()

raw = pd.read_csv("data/inference/AM_ETMM_raw.csv", index_col='id')
raw_labels = raw.apply(extract_label_from_comment, axis=1)

results.submission = replace_predictions(raw_labels, results.prediction)

b = results.submission.copy().tolist()

count = 0
for i in range(len(a)):
    if a[i] != b[i]:
        count += 1
        #print(a[i], b[i])

print(count, 'replaced by label from text')

foto nukk
foto nukk
kiri medal
foto nukk
muusikainstrument raamat
foto raamat
foto raamat
käsikiri nukk
silt/märk pitsat
foto medal
fotonegatiiv dokument
foto kava
lina/linik maal
fotonegatiiv ajakiri
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
graafika joonistus
fotonegatiiv kiri
graafika joonistus
fotonegatiiv raamat
34


In [55]:
# submissions are overwritten predictions
raw['submission'],raw['prediction'], raw['certainty'] = results.submission, results.prediction, results.certainty

In [ ]:
raw.to_csv('data/inference/AM_ETMM_results.csv')

In [72]:
raw['safe'] = raw_labels

In [77]:
safe = raw[raw.safe.notnull()]

In [80]:
np.average(safe.certainty)

0.8756130543976625

In [84]:
len(safe[safe.certainty < 0.5])

30

In [ ]:
# just helps a little

In [67]:
certain = raw[(raw.certainty > 0.5) & (raw.type != 'fotonegatiiv')]

In [68]:
len(certain)

3983

In [75]:
certain.head(1)

,text,additional_text,full_nr,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,ks,collection_queue_nr,collection_additional_nr,name,damages,state,material,commentary,technique,parameter,unit,value,type,certainty
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000763.0,Draamastuudio Teatrikooli aegne foto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12245.0,29.0,NaN,T,232.0,2.0,5,Leo Kalmeti Draamastuudio Teatrikooli aegne foto,NaN,hea,fotopaber,NaN,mustvalge fotograafia,NaN,NaN,NaN,foto,1.483454


In [76]:
raw.tail(2)

,text,additional_text,full_nr,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,ks,collection_queue_nr,collection_additional_nr,name,damages,state,material,commentary,technique,parameter,unit,value,type,certainty,safe
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4309155.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,23085.0,NaN,2,PK,7298.0,NaN,2,"pudel ""Penelope""",NaN,määramata,klaas,NaN,NaN,NaN,NaN,NaN,nõu/anum,0.270446,NaN
4315217.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,25194.0,6.0,NaN,K,8528.0,6.0,NaN,Ratsahobuse varustus: piits,NaN,hea,metall>nahk,,NaN,pikkus,cm,66.5,arheoloogiline leid,0.264965,NaN


In [39]:
significant =  raw[raw.isnull().sum(axis=1)<18]

In [45]:
text = raw[raw.additional_text.notnull()]

In [47]:
text.tail(20)

,text,additional_text,full_nr,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,ks,collection_queue_nr,collection_additional_nr,name,damages,state,material,commentary,technique,parameter,unit,value,type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3879033.0,NaN,Lakkpitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,83,"Diplom des Ehrenmitglieds des Estländischen Gartenbau-Vereins zu Reval, nebst bezüglichem Begleitschreiben",NaN,hea,NaN,NaN,NaN,kõrgus,cm,53.5,foto
3879035.0,NaN,Lakkpitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,85,"Diplom des Ehrenmitglieds der Narvaschen Altertums-Gesellschaft, die bei der Grossen Gilde der See- und Handels-Stadt Narva besteht",NaN,hea,NaN,NaN,NaN,kõrgus,cm,42.5,raamat
3879036.0,NaN,Paberdatud/vahvelpitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,86,"Gratulations-Diplom der Heidelberger Universität, anlässlich des ""Semisaeculariums"" der Doktorwürde",NaN,hea,NaN,NaN,NaN,kõrgus,cm,65.0,raamat
3879038.0,NaN,Paberdatud/vahvelpitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,88,"Gratulations-Diplom der Medizinischen Fakultät bei der Universität Basel, anlässlich der 50-jährigen Jubelfeier der Doktorpromotion",NaN,hea,NaN,NaN,NaN,kõrgus,cm,54.5,raamat
3879043.0,NaN,Reljeefne pitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,93,"Gratulations-Diplom der Bayerischen Akademie der Wissenschaften, anlässlich des 50-jährigen Doktor-Jubiläums",NaN,hea,NaN,NaN,NaN,kõrgus,cm,47.5,raamat
3879045.0,NaN,Reljeefne pitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,95,Diplom des auswärtigen Ehrenmitglieds der Österreichischen Akademie der Wissenschaften,NaN,hea,NaN,NaN,NaN,kõrgus,cm,49.0,raamat
3879046.0,NaN,Reljeefne pitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,96,Диплом почетного члена Вольного экономического общества,NaN,hea,NaN,NaN,NaN,kõrgus,cm,71.0,foto
3879047.0,NaN,Paberdatud/vahvelpitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,97,"Diplom des auswärtigen Mitglieds der Turiner Akademie der Wissenschaften, nebst bezüglichem Begleitschreiben",rebenenud,halb,NaN,NaN,NaN,kõrgus,cm,50.0,raamat
3879050.0,NaN,Paberdatud/vahvelpitser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,NaN,NaN,NaN,NaN,D,50.0,1.0,100,Diplom des korrespondierenden Mitglieds der Erlanger Physisch-Medizinischen Gesellschaft,NaN,hea,NaN,NaN,NaN,kõrgus,cm,26.5,foto


In [40]:
len(significant)

2476

In [43]:
significant.tail(200)

,text,additional_text,full_nr,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,ks,collection_queue_nr,collection_additional_nr,name,damages,state,material,commentary,technique,parameter,unit,value,type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3804229.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13533.0,NaN,NaN,G,4710.0,NaN,NaN,Mehe portree,"määrdunud, niiskusplekid",rahuldav,paber,NaN,litograafia,kõrgus,cm,20.0,graafika
3804236.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13538.0,NaN,NaN,G,4713.0,NaN,NaN,Gerhard von Kügelgeni portree,korras,hea,paber,NaN,ofort,kõrgus,cm,15.6,graafika
3804242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13541.0,NaN,NaN,G,4714.0,NaN,NaN,Carol Morgensterni portree,"määrdunud, nurkades rebendid",rahuldav,paber,NaN,litograafia,kõrgus,cm,26.0,graafika
3804251.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13535.0,NaN,NaN,G,4715.0,NaN,NaN,Carl Friedrich Schoultz,"määrdunud, koltunud",rahuldav,paber,NaN,vasegravüür,kõrgus,cm,24.0,graafika
3804257.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13543.0,NaN,NaN,G,4716.0,NaN,NaN,Friedrich Adolph Philippe,määrdunud,rahuldav,paber,NaN,litograafia,kõrgus,cm,26.0,graafika
3804269.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13542.0,NaN,NaN,G,4717.0,NaN,NaN,Dr. C. F. Senff,määrdunud,rahuldav,paber,NaN,litograafia,kõrgus,cm,34.2,graafika
3804271.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13539.0,NaN,NaN,G,4718.0,NaN,NaN,Dr. A. Hueck,määrdunud,rahuldav,paber,NaN,litograafia,kõrgus,cm,24.7,graafika
3804273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,13511.0,NaN,NaN,G,4719.0,NaN,NaN,Kirumpää Võrumaal,luitunud,rahuldav,paber,NaN,akvarell,kõrgus,cm,14.0,graafika
3804943.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,11884.0,NaN,NaN,T,244.0,1.0,20:03,"Teet Koppel, kaart, Rootsi maanteedekaart, 1947",NaN,hea,paber,NaN,NaN,dokumendi maht,leht,1.0,dokument


In [34]:
raw.head(20)

,text,additional_text,full_nr,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,ks,collection_queue_nr,collection_additional_nr,name,damages,state,material,commentary,technique,parameter,unit,value,type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000763.0,Draamastuudio Teatrikooli aegne foto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12245.0,29.0,NaN,T,232.0,2.0,5,Leo Kalmeti Draamastuudio Teatrikooli aegne foto,NaN,hea,fotopaber,NaN,mustvalge fotograafia,NaN,NaN,NaN,foto
1252943.0,"Nukuteatri nukud: Ruun (""Inimese jälgedes ""Sexappeal"""" 1978)","keha ja pea vineerist, porolooni ja kotiriidega kaetud, kael, lakk takunöörist, loogaks kummivoolik, kaetud pruuni sametiga, aisad värvitud pruuniga vineerplaadid, mis kinnitatud hobuse külge metallvardaga, vanker ühepoolne vineerist, kinnitatud kruvide ja naeltega, juhttraadid, vankri all puidust pulk koos käepidemega, hobuse pea küljes metallist juhtvarras puidust käepidemega",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12293.0,10.0,NaN,Aj,97.0,183.0,T185,Nukk: Ruun Eesti Riikliku Nukuteatri lavastusest Inimese jälgedes: Sexappeal,NaN,hea,puit>samet (riidesort)>metall>looduslik kiud>tekstiil>tehismaterjal>vineer,"pulk vankri all, käepidemed look aisad kinitatud metallvardaga, vankri naelad, kruvid, juhttraadid kael, lakk takunöörist, loogaks kummivoolik keha, pea kaetud kotiriidega poroloon kehas, peas keha, pea, aisad, vanker",NaN,NaN,NaN,NaN,nukk
1252949.0,"Nukuteatri nukud: Kolmas röövel (?)(""Verine John"" 1980)","Juhan Peegel ja Juhan Saar Verine John 1980 Nukuteater. Lavastaja: Ferdinand Veike. Kunstnik: Jaak Vaus. Muusika: Alo Põldmäe. Röövel: Are Uder. Peaks vineer, kaetud porolooni ja puuvillase pruuni soonik sukamaterjaliga. Nina poroloonist, kaetud sukaga. Silmad: alus valgest nahast, sellele kleebitud sinine plastikust ovaalne silmatera, üks silm kaetud mustast sametist klapiga. Kõrvad on poroloonist, kaetud sukaga. Suu käib lahti kummipaelaga, suus kolm poroloonist hammast, kõrvarõngaks metallist kardinarõngas, juuksed pruunist takunöörist. Kaelaks ja kehaks puitpulk kaetud porolooni ja sukaga. Seljas roosa valgete täppidega räbaldunud pluus. Käiste otsas valge sikk-sakk pael. Käed puit, pealt kaetud porolooni ja pruuni sukaga. Sõrmed poroloonist, kaetud valge pehme trikotaažiga. Käed ühendatud juhttraadiga, millel puidust käepidemed. Püksid sinisest puuvillasest riidest",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12293.0,12.0,NaN,Aj,97.0,156.0,T185,Nukk: Kolmas röövel Eesti Riikliku Nukuteatri lavastusest Verine John,NaN,rahuldav,puit>samet (riidesort)>plast>nahk>tehismaterjal,"pulk kehas, käed, juhttraatide käepidemed silmaklapp silma tera silma alus keha, nina porolonn, katteks sukamaterjal",NaN,NaN,NaN,NaN,nukk
1255910.0,"Nukuteatri nukud: Kolentin (""Koll Kalli"" 1985)","pea papjeemašeest, kaetud helepruuni trikotaažiga, suu käib lahti. Peas mustast puuvillasest riidest soni punase nokaga, juuksed plastmassist ribadest, ümmargune esile ulatuv nina,, kaelaks ja kehaks puidust pulk. mis kaetud porolooniga, kaelas villane oranž pruunide plekkidega rätik. Seljas flanellist sinise, valge, pruuni triibuline särk, ees suur nööp, seelikuosa tumesinine villane, kätel puust käepidemetega metallist juhttraadid",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12293.0,38.0,NaN,Aj,97.0,186.0,T185,Nukk: Koll Kalli Eesti Riikliku Nukuteatri lavastusest Koll Kalli,NaN,rahuldav,puit>metall>flanell>papjeemashee,,NaN,pikkus,cm,102.0,nukk
1257288.0,"""Siin me oleme""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12281.0,3.0,NaN,Fk,1401.0,NaN,d,"""Siin me oleme"" (Rež. Sulev Nõmmik).",NaN,hea,NaN,NaN,diapositiiv,laius,mm,36.0,diapositiiv
1267453.0,Kaplinski Õhtu Tallinnas Kirjanike majas. TRÜ näitering.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ETMM,_,12341.0,107.0,NaN,Fk,122262.0,NaN,k,Kaplinski Õhtu Tallinnas Kirjanike maja

In [33]:
raw.tail(200)

,text,additional_text,full_nr,event_type,location,start,end,before_Christ,country_and_unit,participants_role,participant,parish,museum_abbr,musealia_mark,musealia_seria_nr,musealia_queue_nr,musealia_additional_nr,collection_mark,ks,collection_queue_nr,collection_additional_nr,name,damages,state,material,commentary,technique,parameter,unit,value,type
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4302569.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5911.0,NaN,9,PK,519.0,NaN,NaN,Inimese kujuline amulett,NaN,määramata,kips,NaN,NaN,NaN,NaN,NaN,arheoloogiline leid
4302570.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5911.0,NaN,10,PK,520.0,NaN,NaN,Looma kujuline amulett,NaN,määramata,savi,NaN,NaN,NaN,NaN,NaN,arheoloogiline leid
4302571.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5911.0,NaN,11,PK,521.0,NaN,NaN,Inimese kujuline amulett,NaN,määramata,savi,NaN,NaN,NaN,NaN,NaN,arheoloogiline leid
4302572.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5607.0,NaN,1,PK,525.0,NaN,NaN,Kamee,NaN,määramata,kivim,NaN,NaN,NaN,NaN,NaN,arheoloogiline leid
4302573.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5607.0,NaN,2,PK,526.0,NaN,NaN,Kamee,NaN,määramata,kivim,NaN,NaN,NaN,NaN,NaN,arheoloogiline leid
4302574.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5607.0,NaN,3,PK,527.0,NaN,NaN,Kamee,NaN,määramata,kivim,NaN,NaN,NaN,NaN,NaN,arheoloogiline leid
4302575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5877.0,NaN,1,PK,541.0,NaN,01.veebr,Reljeef ja joonis sellest,NaN,määramata,kivim,NaN,NaN,NaN,NaN,NaN,käsikiri
4302582.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5877.0,NaN,2,PK,542.0,NaN,NaN,Papüüruse rulli katke,NaN,määramata,paber,NaN,NaN,NaN,NaN,NaN,käsikiri
4302583.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AM,_,5583.0,NaN,1,PK,553.0,NaN,NaN,Kivimi tükk,NaN,määramata,kivim,NaN,NaN,NaN,NaN,NaN,arheoloogiline leid
